# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.core import Environment,ScriptRunConfig
from azureml.train.estimator import Estimator
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal, choice
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
from train import clean_data
from sklearn.model_selection import train_test_split
from azureml.train.automl import AutoMLConfig

import pandas as pd
import os



## Dataset
TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

experiment_name = 'leukemia_classification'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name,
      'Azure region: ' + ws.location,
      'Subscription id: ' + ws.subscription_id,
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

dataset_path = 'https://raw.githubusercontent.com/PhilippRamjoue/Leukemia_Classification/main/dataset/bone-marrow-dataset.csv'

ds = Dataset.Tabular.from_delimited_files(path=dataset_path)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RL5MZSZCT to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-127504
Azure region: southcentralus
Subscription id: e8f628b3-bb5b-4edf-947a-8637ca6ea7c2
Resource group: aml-quickstarts-127504


In [9]:
# Create compute cluster

cpu_cluster_name = "cluster-leukemia"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',min_nodes=0,
                                                            max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

The model and hyperparameters are explained in the Readme.md file.


In [14]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
        '--C': uniform(0.1, 1.0),
        '--max_iter': choice(10, 25, 50, 100, 1000)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
script_parameter = {
    '--C': 1.0,
    '--max_iter': 100
}

#TODO: Create your estimator and hyperdrive config
estimator = Estimator(source_directory='.',
                script_params=script_parameter,
                compute_target=cpu_cluster,
                entry_script='train.py',
                pip_packages=['sklearn'])

hyperdrive_run_config = HyperDriveConfig(estimator=estimator,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=50,
                             max_concurrent_runs=4)

In [ ]:
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [15]:
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.
The same input parameter(s) are specified in estimator/run_config script params and HyperDrive parameter space. HyperDrive parameter space definition will override these duplicate entries. ['--C', '--max_iter'] is the list of overridden parameter(s).


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [16]:
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

# Save best model
best_run.register_model(model_name='best_model',
                        model_path='outputs/model.joblib')

print('Best Run Id: ', best_run.id)
print(' accuracy:', best_run_metrics['accuracy'])
print(' C:',parameter_values[1])
print(' max iter:',parameter_values[3])


Best Run Id:  HD_f73fc731-2142-4de3-88ec-35fe79667061_0
 accuracy: 0.7878787878787878
 C: 0.7103262339859274
 max iter: 1000


In [18]:
# delete compute cluster
cpu_cluster.delete()

ComputeTargetException: ComputeTargetException:
	Message: Received bad response from Resource Provider:
Response Code: 400
Headers: {'Cache-Control': 'no-cache', 'Pragma': 'no-cache', 'Content-Length': '270', 'Content-Type': 'application/json; charset=utf-8', 'Expires': '-1', 'x-ms-correlation-request-id': 'd9549e35-fb70-41d5-8242-f1b332be9ccf', 'x-ms-ratelimit-remaining-subscription-deletes': '14998', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'x-ms-response-type': 'standard', 'x-ms-request-id': '|00-a0f075cbe00d2142a66653b082ae5fc7-8a678231a2b3914f-00.e693bff0_', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '0.066', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'x-ms-routing-request-id': 'SOUTHCENTRALUS:20201123T090640Z:d9549e35-fb70-41d5-8242-f1b332be9ccf', 'Date': 'Mon, 23 Nov 2020 09:06:40 GMT'}
Content: b'{"error":{"code":"ResourceNotReady","message":"Compute with name cluster-leukemia is already being deleted.","innere

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service